In [1]:
from types import SimpleNamespace
import torch
import os
import json
from torchvision import transforms
from polyvore_outfits import TripletImageLoader
from itertools import chain

gpu = 1

global args
args = SimpleNamespace(
    batch_size = 256,
    seed = 1,
    cuda = torch.device(f"cuda:{gpu}"),
    dim_embed = 64,
    use_fc = False,
    datadir="/home/fteotini/thesis",
    margin=0.3,
    resume="/home/fteotini/thesis/type_aware/runs/Type_Specific_Fashion_Compatibility/model_best.pth.tar",
    polyvore_split="nondisjoint",
    rand_typespaces=False,
    num_rand_embed=4
)

torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

In [2]:
from collections import deque
def ground_truth(outfit):
    l=deque(outfit)
    l.rotate(-1)
    return list(l)

In [3]:
fn = os.path.join(args.datadir, "polyvore_outfits", "polyvore_item_metadata.json")
with open(fn, "r") as fn_file:
    meta_data = json.load(fn_file)

transform = transforms.Compose([
    transforms.Resize(112),
    transforms.CenterCrop(112),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = TripletImageLoader(
    args,
    "test",
    meta_data,
    transform=transform
)

In [4]:
from collections import defaultdict

type2im = defaultdict(list)
for im, type in dataset.im2type.items():
    type2im[type].append(im)

In [5]:
def sample_same_cat_neg(outfit):
    items = torch.as_tensor(list(map(lambda x : int(x),outfit["src"])))
    gr_types = [dataset.im2type[i] for i in outfit['ground_truth']]
    cat_neg = []
    for gr_type in gr_types:
        cand = torch.as_tensor(list(map(lambda x : int(x),type2im[gr_type])))
        prob = torch.ones(len(cand)).float()
        prob[torch.isin(cand,items)] = 0
        ids = torch.multinomial(prob,3)
        cat_neg.append(list(map(lambda x: str(x),cand[ids].tolist())))

    return cat_neg

In [6]:
def sample_neg(outfit):
    items_id = torch.as_tensor([dataset.im2index[i] for i in outfit["src"]])
    cand = torch.as_tensor(list(dataset.im2index.values()))
    prob = torch.ones(len(cand)).float()
    prob[items_id] = 0
    idx = torch.multinomial(prob, len(items_id)*5).view(-1,5)
    values = torch.as_tensor(list(map(lambda x : int(x),dataset.im2index.keys())))[cand[idx]]
    return list(map(lambda x: list(map(lambda y: str(y),x)),values.tolist()))

In [7]:
rootdir = os.path.join(args.datadir, "polyvore_outfits", args.polyvore_split)
with open(os.path.join(rootdir, "%s.json" % 'test'), "r") as data_json:
    outfit_data = json.load(data_json)

In [8]:
outfits = [list(map(lambda x: x['item_id'], i['items'])) for i in outfit_data]
outfits = [{
    'src': o,
    'ground_truth': ground_truth(o)
} for o in outfits]
outfits = [{**o, 'negs': [list(chain(*l)) for l in zip(sample_same_cat_neg(o),sample_neg(o))]} for o in outfits]

In [9]:
import pickle

with open("sip_data_nondisjoint.pkl",'wb') as f:
    pickle.dump(outfits,f,pickle.HIGHEST_PROTOCOL)